In [ ]:
import qutip
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import project_lib as mylib

In [ ]:
qubits = 4
points = 1000
T = 100.
dt = T/points
t = np.linspace(0,T,points)
h = np.random.random(qubits)
J = np.random.random((qubits,qubits))

In [ ]:
a = mylib.Anneal(qubits,[h,J],T=T,points = points, light = False)
a.run()

In [ ]:
def hamiltonian_parts(h,J):
        '''
        - Constructs the hamiltonian from the parameters input
        - If we are doing an anneal run where we have different annealing schedules for each qubits it will return a list 
        of hamiltonian parts for the base hamiltonian. Else it returns a single hamiltonian for this part
        '''
        qubits = len(h)
        #initiate the hamiltonians giving them the right dimensions
        Hb = qutip.tensor([qutip.identity(2) for i in range(qubits)])*0
        Hp = qutip.tensor([qutip.identity(2) for i in range(qubits)])*0
        # loop through adding pieces to construct the hamiltonians 
        for i in range(qubits):
            Hb += qutip.tensor([qutip.identity(2) for n in np.arange(i)]+[qutip.sigmax()]+[qutip.identity(2) for n in np.arange(qubits-i-1)])
            Hp += h[i]*qutip.tensor([qutip.identity(2) for n in np.arange(i)]+[qutip.sigmaz()]+[qutip.identity(2) for n in np.arange(qubits-i-1)])
            # the following part adds the interaction between all qubits. There is 1 interaction coefficient for each pairing 
            for j in range(i+1,qubits): 
                sigz_i = qutip.tensor([qutip.identity(2) for n in np.arange(i)]+[qutip.sigmaz()]+[qutip.identity(2) for n in np.arange(qubits-i-1)])
                sigz_j = qutip.tensor([qutip.identity(2) for n in np.arange(j)]+[qutip.sigmaz()]+[qutip.identity(2) for n in np.arange(qubits-j-1)])
                Hp +=J[i][j]*sigz_i*sigz_j  
 
        return Hb, Hp

# in the computational basis and what they seem to use in the qutip package
one = qutip.basis(2, 0)
zero = qutip.basis(2, 1)
basis = [zero,one]

from tabulate import tabulate
def qu_bit_table(Qo,qubits):
    '''given a state this fucntion prints out the table of the probability of measuring this state in it's eigenfunctions'''
    headers = ['Bit Solution','Final State Probability']
    ket = Qo.states[-1]
    bits = []
    last_state_probs = []
    table = []
    for i in range(2**qubits):
        bits.append( [(i/2**j)%2 for j in range(qubits)[::-1]])
        bra = (qutip.tensor([one if i ==1 else zero for i in bits[-1]])).dag()
        last_state_probs.append((abs((bra*ket)[0,0])**2))
        table.append([bits[-1],last_state_probs[-1]])
    
    table.insert(0,['','Full Results',''])
    #table.append(['Total_Prob - 1: ',float(np.sum(last_state_probs)-1),''])
    print tabulate(table, headers, tablefmt="grid")


####################################################################################
# FORMULATION NEEDED FOR QUTIP
####################################################################################
def A(t,args):
    return 1-t/T
def B(t,args): 
    return t/T

Hb, Hp = hamiltonian_parts(h,J)

ws = 2**(-0.5)*(qutip.basis(2, 1)+qutip.basis(2, 0))
psi0 = qutip.tensor([ws for n in np.arange(qubits)])

bra1000 = (qutip.tensor([one]+[zero]*3)).dag()
ket1000 = qutip.tensor([one]+[zero]*3)
proj1000 = ket1000*bra1000

c_ops = []
e_ops = [proj1000]


Options = qutip.solver.Options(store_states=True)
H0 = Hb*0.
H = [H0, [Hb, A], [Hp, B]]
qu = qutip.mesolve(H, psi0, t,c_ops,e_ops=e_ops,options = Options)
qu_bit_table(qu,qubits)

In [ ]:
print qutip.expect(Hp, qutip.tensor([one]+[zero]*3)), #(np.dot(a.states[-1].getH(),a.Hp*a.states[-1])).item(0)

In [ ]:
print qutip.expect(Hb, qutip.tensor([ws for n in np.arange(qubits)])), (np.dot(a.states[0].getH(),a.Hb*a.states[0])).item(0)

In [ ]:
atab = mylib.bit_table(a.states[-1],a.problem_x0s, a.qubits)

In [ ]:
zero = qutip.basis(2, 0) # up
one = qutip.basis(2, 1) # down
basis = [zero,one]
qutip.tensor([ws for n in np.arange(qubits)]) 

In [ ]:
print one.__dict__.keys()

In [ ]:
(qutip.bra([0])*qutip.ket([0])).__dict__.keys()

In [ ]:
bra1000*proj1000*ket1000

In [ ]:
print qu.__dict__.keys()
print qu.expect